In [1]:
# import nltk
import numpy as np
import pandas as pd
from time import time
import pickle
import re

from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

from nltk.corpus import stopwords
import pymorphy2
from nltk.stem import WordNetLemmatizer
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2

In [1]:
X_train = pickle.load(open('/Users/usual/PycharmProjects/npl_project01/data/X_train.pckl', 'r'))
y_train = pickle.load(open('/Users/usual/PycharmProjects/npl_project01/data/y_train.pckl', 'r'))
X_test = pickle.load(open('/Users/usual/PycharmProjects/npl_project01/data/X_test.pckl', 'r'))

NameError: name 'pickle' is not defined

In [25]:
def predicting(clf, clf_name, x_train, y_train, x_test, y_test):
    # clf = MultinomialNB(alpha=.01)  # BernoulliNB(alpha=.01)
    print '_' * 80
    print "%s" % clf_name
    print "Training:"
    t0 = time()
    clf.fit(x_train, y_train)
    train_time = time() - t0
    print "train time: %0.3fs" % train_time

    print "Predicting: "
    t0 = time()
    pred = clf.predict(x_test)
    test_time = time() - t0
    print "predicting time:  %0.3fs" % test_time
    print "Predicting length: %d" % len(pred)
    for uid, pred_f in zip(y_test.tolist(), pred):
        print uid, pred_f

    score = metrics.accuracy_score(y_test, pred)
    print "accuracy:   %0.3f" % score
    # print pred

    print "classification report:"
    # print uids_test[0]
    print metrics.classification_report(y_test, pred)  # , target_names=uids_test

    print "confusion matrix:"
    print metrics.confusion_matrix(y_test, pred)

In [7]:
X_train.head()

,1080p,16gb,1xe,24open,720p,access,accessori,acer,across,action,...,япония,японка,японский,яркий,ярко,ярославль,яценюк,ящик,ёлка,gender
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [8]:
y_train

0        0
1        0
2        0
3        1
4        1
5        2
6        1
7        3
8        0
9        4
10       4
11       2
12       3
13       2
14       4
15       4
16       2
17       4
18       4
19       3
20       4
21       4
22       4
23       4
24       2
25       2
26       2
27       4
28       4
29       1
        ..
34312    1
34313    2
34314    2
34315    2
34316    4
34317    0
34318    4
34319    0
34320    2
34321    1
34322    4
34323    1
34324    1
34325    4
34326    4
34327    4
34328    2
34329    4
34330    2
34331    1
34332    0
34333    4
34334    2
34335    4
34336    2
34337    2
34338    0
34339    1
34340    4
34341    2
Name: age, dtype: int64